In [1]:
import os
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
import nltk
import pprint
import pdb
import re
import traceback
import pandas as pd

resume_dir = '/net/data/hiring-sites/indeed/resumes/'
data_dir   = '/net/data/hiring-sites/indeed/data/'

In [2]:
'''
Creating embeddings 

Features
-  Full Name : Use a char-embedding approach [race/gender]
-  additionalInfo : (Glove embeddings of each word [mean]/[max]) / Bert encoder -> [sentence level representation] 
-  eductaion : One-hot-encoding of tuples with (degree, school)
-  headline : same tech as additionalInfo
-  lastUpdated : store as unix representation of date
-  workExperience : Tuples of company, description, customized date range, current
-  

'''

'\nCreating embeddings \n\nFeatures\n-  Full Name : Use a char-embedding approach [race/gender]\n-  additionalInfo : (Glove embeddings of each word [mean]/[max]) / Bert encoder -> [sentence level representation] \n-  eductaion : One-hot-encoding of tuples with (degree, school)\n-  headline : same tech as additionalInfo\n-  lastUpdated : store as unix representation of date\n-  workExperience : Tuples of company, description, customized date range, current\n-  \n\n'

In [3]:
import sys; sys.path.append('FairRanking/')
from gender_classifier import *
from ethnicity_classifier import *

gender_dict1 = {}
gender_dict1['seed']       = 100
gender_dict1['classifier'] = 'gender'
gender_dict1['middle_name']= 'no'

ethnicity_dict1 = {}
ethnicity_dict1['seed']       = 100
ethnicity_dict1['classifier'] = 'ethnicity'
ethnicity_dict1['middle_name']= 'no'

gender_dict2 = {}
gender_dict2['seed']       = 11747
gender_dict2['classifier'] = 'gender'
gender_dict2['middle_name']= 'no'

ethnicity_dict2 = {}
ethnicity_dict2['seed']       = 11747
ethnicity_dict2['classifier'] = 'ethnicity'
ethnicity_dict2['middle_name']= 'no'

gender_dict3 = {}
gender_dict3['seed']       = 42
gender_dict3['classifier'] = 'gender'
gender_dict3['middle_name']= 'no'

ethnicity_dict3 = {}
ethnicity_dict3['seed']       = 42
ethnicity_dict3['classifier'] = 'ethnicity'
ethnicity_dict3['middle_name']= 'no'

from collections import defaultdict as ddict

ethnic_map_dict = {}
ethnic_map_dict['Latino']     =    'Hispanic'
ethnic_map_dict['White']      =    'Caucasian'
ethnic_map_dict['Asian']      =    'Asian'
ethnic_map_dict['Black']      =    'African-American'
ethnic_map_dict['NativeAm']   =    'Native American'

gender_map_dict = {}
gender_map_dict['M'] = 'Male'
gender_map_dict['F'] = 'Female'


def maj_vote(name):
    eth_vals = ddict(int)
    for edict in [ethnicity_dict1, ethnicity_dict2, ethnicity_dict3]:
        v = call_ethnicity_inference(name, edict)
        eth_vals[v]+=1
    race  = max(eth_vals, key=eth_vals.get)

    gen_vals = ddict(int)
    for gdict in [gender_dict1, gender_dict2, gender_dict3]:
        v = call_gender_inference(name, gdict)
        gen_vals[v]+=1

    gender = max(gen_vals, key=gen_vals.get)    
    return ethnic_map_dict[race]+'_'+gender_map_dict[gender]


def maj_vote_batch(names):
    eth_vals = [ddict(int) for name in names]
    
    for edict in [ethnicity_dict1, ethnicity_dict2, ethnicity_dict3]:
        vals = call_ethnicity_inference_batch(names, edict)
        for i, v in enumerate(vals):
            eth_vals[i][v]+=1
    
    races  = [max(eth_vals[i], key=eth_vals[i].get) for i in range(len(eth_vals))]

    gen_vals = [ddict(int) for name in names]
    
    for gdict in [gender_dict1, gender_dict2, gender_dict3]:
        vals = call_gender_inference_batch(names, gdict)
        for i, v in enumerate(vals):
            gen_vals[i][v]+=1

    genders = [max(gen_vals[i], key=gen_vals[i].get) for i in range(len(gen_vals))]   
    
    return [ethnic_map_dict[race]+'_'+gender_map_dict[gender] for race, gender in zip(races, genders)]





7722297it [01:02, 124071.06it/s]


{'M': 0, 'F': 1}


7722297it [01:02, 123631.05it/s]


In [7]:
import torch
torch.cuda.is_available() 

False

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

maj_vote('ritam')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [6]:
resume_list = os.listdir(resume_dir) 
cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def preprocess_text(text):
	cleantext = re.sub(cleanr, '', text)
	return cleantext

from joblib import Parallel, delayed
import logging
import itertools
# import dill as pickle
from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass

def partition(lst, n):
    if n == 0: return lst
    division = len(lst) / float(n)
    return [ lst[int(round(division * i)): int(round(division * (i + 1)))] for i in range(n) ]

def get_chunks(inp_list, chunk_size):
    return [inp_list[x:x+chunk_size] for x in range(0, len(inp_list), chunk_size)]

def mergeList(list_of_list):
    return list(itertools.chain.from_iterable(list_of_list))


data            =   [page for page in resume_list]

num_procs       =   5
chunks          =   partition(data, num_procs)

print(num_procs, len(data), len(chunks))

def create_df(pid, datalist):
    print('Starting for pid {} on {}'.format(pid, len(datalist)))
    resume_dir = '/net/data/hiring-sites/indeed/resumes/'
    resume_dict = {}
    resume_dict['queryname'] =   []
    resume_dict['name']  =   []
    resume_dict['race+gender']    =   []
    resume_dict['rank']      =   []

    for count, resume_page in enumerate(datalist):
        resume_arr = open(resume_dir+resume_page).read().split('\n')
        
        for resume_str in resume_arr:
            if resume_str == '':
                continue
            try:
                resumes = json.loads(preprocess_text(resume_str))
            except Exception as e:
                continue
            
            full_names   = [resume['displayFullName'] for resume in resumes]
            text_names   = [re.sub(r'[^a-z ]+','',name.lower()).strip() + ' '*max(0,5-len(re.sub(r'[^a-z ]+','',name.lower()).strip()))  for name in full_names]
            rcs          = maj_vote_batch(text_names)
            ranks        = [i+1 for i in range(len(full_names))]
            
            resume_dict['queryname'].extend([resume_page for i in range(len(full_names))])
            resume_dict['name'].extend(full_names)
            resume_dict['race+gender'].extend(rcs)
            resume_dict['rank'].extend(ranks)
            
            for resume in resumes:
                try:
                    name = resume['displayFullName']                    
                    race_gender = maj_vote(re.sub(r'[^a-z ]+','',name.lower()).strip())
#                     print(name, race_gender)
                    resume_dict['queryname'].append(resume_page)
                    resume_dict['name'].append(name)
                    resume_dict['race+gender'].append(race_gender)
                    resume_dict['rank'].append(rank)
                    rank += 1
                except Exception as e:
#                     print(e)
                    continue
        
        print("Done {}/{} for pid {}".format(count, len(datalist),pid), end= '\r')
    
    
    df = pd.DataFrame(resume_dict)
    display(df.head())
    
    return [df]

resume_df_arr            =   create_df(0,data)

# resume_df_arr          =   mergeList(Parallel(n_jobs = num_procs, backend='multiprocessing')(delayed(create_df)(i, chunk) for i, chunk in enumerate(chunks)))

consolidated_resume_df =   pd.concat(resume_df_arr)

display(consolidated_resume_df.head())



5 6874 5
Starting for pid 0 on 6874


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [18]:
consolidated_resume_df.to_csv('indeed_resume.csv')

In [ ]:
resume_dict = {}
resume_dict['queryname'] =   []
resume_dict['fullname']  =   []
resume_dict['gender']    =   []
resume_dict['race']      =   []
resume_dict['rank']      =   []


for count, resume_page in enumerate(resume_list):
    if count >3:
        break
    resume_arr = open(resume_dir+resume_page).read().split('\n')
    rank       = 1
    for resume_str in resume_arr:
        try:
            print('\n\n')
            if resume_str =='':
                continue
            resumes = json.loads(preprocess_text(resume_str))
            for resume in resumes:
                name = resume['displayFullName']
                
        except Exception as e:
              print(e)
#             traceback.print_exc()
            
            

In [7]:
! which pip

/home/avijit/anaconda3/envs/py37/bin/pip


In [8]:
nc_dir = '/home/piotr/voter-records-nc/'

In [9]:
import csv
import numpy as np
import zipfile
from tqdm import tqdm

In [34]:
edic = {}
gdic = {}
def load(infile):
        races = set([])
        eths = set([])
        data = []
        with open(infile, encoding='utf-8', errors='ignore') as csvfile:
            reader = csv.reader(csvfile, delimiter = '\t', quotechar='"')
            first = True
            for row in tqdm(reader):
                if first:
                    first = False
                    continue

                last = row[9]
                first = row[10]
                middle = row[11]
                city = row[14]
                state = 'NC'
                zipcode = row[16]
                race = row[25]
                races.add(race)
                ethnic = row[26]
                eths.add(ethnic)
                gender = row[28]
                status = row[4]
                party = row[27]
                age = row[29]
                if race+'_'+ethnic not in edic:
                    edic[race+'_'+ethnic]=[]
                edic[race+'_'+ethnic].append(first+" "+middle+" "+last)
                if gender not in gdic:
                    gdic[gender]=[]
                gdic[gender].append(first+" "+middle+" "+last)
                
                data.append((first, middle, last, city, state, zipcode, race, ethnic, party, age, gender))

        print(races)
        print(eths)

        return data

In [35]:
data = load(nc_dir+'ncvoter_Statewide.txt')


0it [00:00, ?it/s]
11816it [00:00, 118159.83it/s]
23073it [00:00, 116420.31it/s]
34614it [00:00, 116114.48it/s]
46319it [00:00, 116392.93it/s]
57878it [00:00, 116148.75it/s]
68962it [00:00, 114501.74it/s]
80003it [00:00, 113229.47it/s]
91339it [00:00, 113268.39it/s]
102591it [00:00, 113042.45it/s]
113997it [00:01, 113345.27it/s]
125853it [00:01, 114859.64it/s]
137416it [00:01, 115089.62it/s]
149621it [00:01, 117091.27it/s]
161228it [00:01, 116271.29it/s]
173028it [00:01, 116783.60it/s]
185372it [00:01, 118698.47it/s]
197878it [00:01, 120537.37it/s]
209940it [00:01, 120561.15it/s]
221988it [00:01, 117943.23it/s]
233790it [00:02, 116515.11it/s]
245451it [00:02, 115625.55it/s]
257022it [00:02, 114372.93it/s]
268468it [00:02, 113057.03it/s]
279934it [00:02, 113530.25it/s]
291295it [00:02, 111704.84it/s]
302478it [00:02, 110057.85it/s]
313498it [00:02, 108559.41it/s]
324369it [00:02, 107784.71it/s]
335324it [00:02, 108308.09it/s]
346164it [00:03, 106729.70it/s]
356849it [00:03, 106454.92it

5040575it [00:46, 113215.60it/s]
5051910it [00:47, 112604.88it/s]
5063706it [00:47, 114159.61it/s]
5075192it [00:47, 114368.37it/s]
5087078it [00:47, 115676.14it/s]
5098656it [00:47, 115494.15it/s]
5110213it [00:47, 114988.41it/s]
5121730it [00:47, 115027.90it/s]
5133661it [00:47, 116268.11it/s]
5145294it [00:47, 114292.94it/s]
5157048it [00:47, 115245.91it/s]
5168899it [00:48, 116180.87it/s]
5180574it [00:48, 116350.40it/s]
5192216it [00:48, 113505.84it/s]
5203587it [00:48, 113189.52it/s]
5214983it [00:48, 113415.48it/s]
5226517it [00:48, 113985.04it/s]
5238203it [00:48, 114826.95it/s]
5249693it [00:48, 114586.16it/s]
5261157it [00:48, 114107.67it/s]
5272774it [00:48, 114717.70it/s]
5284531it [00:49, 115558.29it/s]
5296091it [00:49, 115212.13it/s]
5307930it [00:49, 116145.75it/s]
5319549it [00:49, 115274.27it/s]
5331081it [00:49, 112917.58it/s]
5342387it [00:49, 110981.20it/s]
5353503it [00:49, 109325.96it/s]
5364454it [00:49, 108818.91it/s]
5375349it [00:49, 106995.78it/s]
5386065it 

{'O', 'U', ' ', 'B', 'A', 'M', 'W', 'I'}
{'UN', 'HL', 'NL'}


In [37]:
gdic['U']

['BRANDON ISAC ABERCROMBIE',
 'JACOBI SANCHEZ ABRUN',
 'JOSEPH LYNN ADAMS',
 'LARRY DARNELL ADAMS',
 'RICHARD ALEXANDER ADAMS',
 'SHAUNA RENEE ADAMS',
 'ANDREW WILLIAM ADCOCK',
 'ADEPOJU OLADIPUPO ADEOTI',
 'LETISHA BUSH ADKINS',
 'IMRAN  AFZAL',
 'AJ J AGUILAR',
 'CARA HICKS AGUNDIS',
 'SHERMAN ROY AIKALA',
 'FORREST LAFAYETTE AIKEN',
 'TIMOTHY  AKERS',
 'DOUGLAS BERNARD AKINS',
 'KELLY ANN ALBRIGHT',
 'WILLIAM ANTONIO ALBRIGHT',
 'JOSHUA WAYNE ALCON',
 'LOIS BUTLER ALDRIDGE',
 'KELLY MARIE ALFORD',
 'MARK BYRON ALFORD',
 'BRANDON MICHAEL ALLEN',
 'CHARISMA E ALLEN',
 'JAMIE L ALLEN',
 'KONNESHA I ALLEN',
 'SHARON JEFFRIES ALLEN',
 'SANDRA CAGLE ALLEN-MORTON',
 'LUTHER ONEAL ALLISON',
 'DEATRA FOSTER ALLRED',
 'AMANDA MARIE ALSTON',
 'CORNISA MARIE ALSTON',
 'EBONY KIERA ALSTON',
 'KELLY ANN ALSTON',
 'SAUNDERS CLARENCE ALSTON',
 'MIANA JOY ALVARADO',
 'ROBERT ANDREW ALVARADO',
 'CLAUDIA  ALVAREZ',
 'JEFFERSON  ALVAREZ',
 'CARLY RENEE ALVES',
 'DAVID ULADIMIR AMAYA',
 'TRINITY KATHRYN

In [39]:
edic.keys()

dict_keys(['W_NL', 'W_UN', 'W_HL', 'O_NL', 'A_UN', 'B_NL', 'B_UN', 'O_UN', 'A_NL', 'U_UN', 'O_HL', 'M_NL', 'U_HL', 'M_UN', 'I_NL', 'U_NL', 'M_HL', 'A_HL', 'B_HL', 'I_UN', 'I_HL', ' _UN'])

In [44]:
edic['A_NL']

['JOHN FUJIO ABE',
 'ELSIE JOHN ABRAHAM',
 'ANALYN ESTOQUE ABULOC',
 'MAYANK JITENDRA ACHARYA',
 'TAMARA LEE ADIBI',
 'ARASI SWAMICKANNU ADKINS',
 'NISARAHMAD  AKBARI',
 'HONG VINH AU',
 'NAI KO KO AUNG',
 'HAMDA JABBAR AWAN',
 'DAVID THIEN BANH',
 'FELIX CANTO BASCO',
 'VENON CANTO BASCO',
 'CHARRO MAY GARCIA BAUZON',
 'ALEXIS KEHAULANI BERGMANN',
 'LA  BOUNHIENG',
 'TITH  BOUNHIENG',
 'MA CHARINA ILLESCAS BROOKS',
 'SUSAN LEE BROWN',
 'DIEP THI BUI',
 'KIM DUNG BUI',
 'NHAN T BUI',
 'SAMANTHA DANPHUONG BUTLER',
 'EDGAR SALVADOR JR ROMENA BUYCO',
 'WILMAR CANO CABUANG',
 'XIVLAN  CAO',
 'BONN JORGE GONZALES CARVAJAL',
 'HSIU-NIEN  CHANG',
 'JOHN HANGSUK CHANG',
 'MOHAMMAD TAYYAB CHAUDHRY',
 'GULZARA SINGH CHEEMA',
 'FENCHENG  CHEN',
 'MISBAH NASIR CHHOTANI',
 'RIZWANA NASIR CHHOTANI',
 'TAMA  CLARKE',
 'SPENCER JOSEPH-PAUL COFFEY',
 'PAMELA LO COLM',
 'KATHERINE MEICONG CONLIN',
 'BUNYANECH DAWANGPA CORBETT',
 'SHANEE DAVID COWLAND',
 'ZAITOON YASMIN DADA',
 'AMEE TANUJ DADHANIA',
 'G

In [ ]:
# Name ethnicity 